In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

# load mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Tensorflow random angle rotation
input_size = mnist.train.images.shape[1]
side_size = int(np.sqrt(input_size))

dataset = tf.placeholder(tf.float32, [None, input_size])
images = tf.reshape(dataset, (-1, side_size, side_size, 1))
random_angles =tf.cast(tf.random.uniform(shape = (tf.shape(images)[0], ), \
                                  minval = 0, maxval = 3, dtype = tf.int32) , dtype=tf.float32) *(np.pi / 2)

rotated_images = tf.contrib.image.transform(
    images,
    tf.contrib.image.angles_to_projective_transforms(
        random_angles, tf.cast(tf.shape(images)[1], tf.float32), tf.cast(tf
            .shape(images)[2], tf.float32)
    ))

# Run and Print
sess = tf.Session()
result = sess.run(rotated_images, feed_dict = {
    dataset: mnist.train.images,
})

original = np.reshape(mnist.train.images * 255, (-1, side_size, side_size)).astype(
    np.uint8)
rotated = np.reshape(result * 255, (-1, side_size, side_size, 1)).astype(np.uint8)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Lambda
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Layer

class LocalResponseNormalization(Layer):

    def __init__(self, n=5, alpha=1e-4, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        _, r, c, f = self.shape 
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1,1), padding="same", pool_mode='avg')
        summed = K.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom 
    
    def compute_output_shape(self, input_shape):
        return input_shape 
input_shape = (28, 28, 1)

model = Sequential()
model.add(Conv2D(24, (11, 11), padding='same', input_shape=input_shape))
model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
model.add(LocalResponseNormalization(input_shape=model.output_shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

model.add(Conv2D(96, (3, 3), activation='relu', padding='same'))
model.add(LocalResponseNormalization(input_shape=model.output_shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

model.add(Conv2D(96, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

optimizer = optimizers.SGD(lr=0.01, decay=5e-5, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 24)        2928      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        38464     
_________________________________________________________________
local_response_normalization (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 96)        55392     
_________________________________________________________________
local_response_normalization (None, 13, 13, 96)        0         
_________________________________________

In [ ]:
batch = 20
epochs = 5

class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

hist = model.fit(rotated, mnist.train.labels, epochs=epochs, batch_size = 1000)
'''loss, acc = model.evaluate(test_images, test_labels)

plt.figure()
plt.plot(acc)
plt.show()'''

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 55000 samples
Epoch 1/5
44000/55000 [=======================>......] - ETA: 5:32 - loss: 2.4529 - acc: 0.1030

In [ ]:
hist = model.fit(rotated, mnist.train.labels, epochs=5, batch_size = 1000)

Train on 55000 samples
Epoch 1/5


In [57]:
test_loss, test_acc = model.evaluate(mnist.test.images.reshape(-1, 28,28,1) ,  mnist.test.labels, verbose=2)

print('\n테스트 정확도:', test_acc)

10000/10000 - 24s - loss: 2.3018 - acc: 0.1135

테스트 정확도: 0.1135


In [58]:
predictions = model.predict(test_images)

NameError: name 'test_images' is not defined

In [16]:
predictions[0]

array([8.7407176e-13, 4.4898311e-08, 3.4882508e-08, 7.8317271e-08,
       1.7498726e-08, 1.8597500e-13, 3.0422227e-19, 9.9999964e-01,
       3.1765839e-09, 1.8099250e-07], dtype=float32)

In [ ]:
# Run
sess = tf.Session()
result = sess.run(rotated_images, feed_dict = {
    dataset: mnist.train.images,
})

original = np.reshape(mnist.train.images * 255, (-1, side_size, side_size)).astype(
    np.uint8)
rotated = np.reshape(result * 255, (-1, side_size, side_size)).astype(np.uint8)



# Print 10 random samples
'''fig, axes = plt.subplots(2, 10, figsize = (15, 4.5))
choice = np.random.choice(range(len(mnist.test.labels)), 10)
for k in range(10):
    axes[0][k].set_axis_off()
axes[0][k].imshow(original[choice[k, ]], interpolation = 'nearest', \
    cmap = 'gray')
axes[1][k].set_axis_off()
axes[1][k].imshow(rotated[choice[k, ]], interpolation = 'nearest', \
    cmap = 'gray')'''

In [26]:
a = 


tf.Session().run(a)

array([3.1415927, 1.5707964, 0.       , 0.       , 0.       , 1.5707964,
       0.       , 3.1415927, 1.5707964, 0.       ], dtype=float32)